# Quora insincere Question Classification

Kaggle - https://www.kaggle.com/c/quora-insincere-questions-classification

In [1]:
import tensorflow as tf
print(tf.version.VERSION)

2.7.0


In [2]:
!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 2650, done.
remote: Counting objects: 100% (2650/2650), done.
remote: Compressing objects: 100% (2311/2311), done.
remote: Total 2650 (delta 505), reused 1389 (delta 306), pack-reused 0
Receiving objects: 100% (2650/2650), 34.02 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (505/505), done.
Note: switching to '400d68abbccda2f0f6609e3a924467718b144233'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

Updating files: 100% (2554/2554), done.


In [2]:
# install requirements to use tensorflow/models repository
!pip install -Uqr models/official/requirements.txt
# you may have to restart the runtime afterwards

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
from official.nlp import optimization

/Users/a.daggula/opt/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [4]:
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TF Version:  2.7.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


A downloadable copy of the Quora Insincere Questions Classification data can be found https://archive.org/download/fine-tune-bert-tensorflow-train.csv/train.csv.zip. Decompress and read the data into a pandas DataFrame.

In [3]:
dataset = '/Users/a.daggula/Downloads/train.csv'

In [18]:
df = pd.read_csv(dataset,nrows=10000)

In [19]:
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [20]:
df['target'].value_counts()

0    9363
1     637
Name: target, dtype: int64

### Split the train and test datasets

In [23]:
train_df, remaining = train_test_split(df, random_state=42, train_size=0.70, stratify=df.target.values)
valid_df, _ = train_test_split(remaining, random_state=42, train_size=0.60, stratify=remaining.target.values)

In [24]:
train_df.shape, valid_df.shape

((7000, 3), (1800, 3))

In [25]:
train_df['target'].value_counts(), valid_df['target'].value_counts()

(0    6554
 1     446
 Name: target, dtype: int64,
 0    1685
 1     115
 Name: target, dtype: int64)

In [27]:
with tf.device('/cpu:0'):
    train_data = tf.data.Dataset.from_tensor_slices((train_df['question_text'].values, train_df['target'].values))
    valid_data = tf.data.Dataset.from_tensor_slices((valid_df['question_text'].values, valid_df['target'].values))
    
    for text, label in train_data.take(1):
        print(text,label)
    

tf.Tensor(b"What's the route to become a professional football player in India?", shape=(), dtype=string) tf.Tensor(0, shape=(), dtype=int64)


#### Downlaod Pre-trained Bert Model from Tensorflow Hub

Get BERT layer and tokenizer:               
More details here: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2

In [30]:
labels = [0, 1] # Target classes
max_seq_length = 64 # Maximum seq length of the question
train_batch_size = 16

In [33]:
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2', trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

#### Tokenize and Process Data for BERT

In [34]:
def text_to_feature(text, label, 
                    label_list=labels, 
                    max_sequence_length=max_seq_length,
                    tokenizer=tokenizer):
    example = classifier_data_lib.InputExample(guid=None,
                                              text_a = text.numpy(),
                                              text_b = None,
                                              label = label.numpy())
    feature = classifier_data_lib.convert_single_example(0,example, label_list, max_seq_length, tokenizer)
    
    return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

#### Wrap a Python Function into a Tensorflow op for Eager Execution

In [41]:
def text_to_feature_map(text,label):
    input_ids, input_mask, segment_ids, label_id = tf.py_function(text_to_feature, inp=[text, label],
                                                                 Tout=[tf.int32,tf.int32,tf.int32,tf.int32])
    input_ids.set_shape([max_seq_length])
    input_mask.set_shape([max_seq_length])
    segment_ids.set_shape([max_seq_length])
    label_id.set_shape([])
    
    x = {
        'input_word_ids': input_ids,
        'input_mask': input_mask,
        'input_type_ids': segment_ids,
    }
    return (x,label_id)

#### Create a Tensorflow Input Pipeline with tf.data

In [42]:
with tf.device('/cpu:0'):
    #train 
    train_data = (train_data.map(text_to_feature_map,num_parallel_calls=tf.data.experimental.AUTOTUNE)
                  .shuffle(1000)
                  .batch(32, drop_remainder=True)
                  .prefetch(tf.data.experimental.AUTOTUNE))
    #valid 
    valid_data = (valid_data.map(text_to_feature_map,num_parallel_calls=tf.data.experimental.AUTOTUNE)
                  .shuffle(1000)
                  .batch(32, drop_remainder=True)
                  .prefetch(tf.data.experimental.AUTOTUNE))

In [43]:
#train data spec
train_data.element_spec

({'input_word_ids': TensorSpec(shape=(32, 64), dtype=tf.int32, name=None),
  'input_mask': TensorSpec(shape=(32, 64), dtype=tf.int32, name=None),
  'input_type_ids': TensorSpec(shape=(32, 64), dtype=tf.int32, name=None)},
 TensorSpec(shape=(32,), dtype=tf.int32, name=None))

In [44]:
#valid data spec
valid_data.element_spec

({'input_word_ids': TensorSpec(shape=(32, 64), dtype=tf.int32, name=None),
  'input_mask': TensorSpec(shape=(32, 64), dtype=tf.int32, name=None),
  'input_type_ids': TensorSpec(shape=(32, 64), dtype=tf.int32, name=None)},
 TensorSpec(shape=(32,), dtype=tf.int32, name=None))

### Add Classification Head to Bert Layer

In [52]:
def create_model():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])
    
    drop = tf.keras.layers.Dropout(0.4)(pooled_output)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(drop)
    
    model = tf.keras.Model(
        inputs = {
            'input_word_ids':input_word_ids,
            'input_mask':input_mask,
            'input_type_ids':input_type_ids
        },
        outputs=output)
    return model

#### Fine Tune the Model

In [53]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
             loss = tf.keras.losses.BinaryCrossentropy(),
             metrics = [tf.keras.metrics.BinaryAccuracy()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 64)]         0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 64, 768)]                 'input_mask[0][0]',         

In [57]:
# tf.keras.utils.plot_model(model=model, show_shapes=True, dpi=76)

In [59]:
epochs = 1
history = model.fit(train_data, 
                    validation_data=valid_data,
                    epochs= epochs,
                    verbose=1)

218/218 [==============================] - 40015s 184s/step - loss: 0.1842 - binary_accuracy: 0.9339 - val_loss: 0.1372 - val_binary_accuracy: 0.9487
